
### Running MongoDB

- exam version mongodb 3.4.9, pymongo 3.6.0 (found in gdrive A-level software)

- shortcut on Desktop (Default and ONLY way on exam machines)
- start menu MongoDB Server
    - (*default database is on user's profile AppData\Local\Temp*)

- run command "mongod --dbpath=zzz", where zzz is a read/write accesible path
- run as a Windows Service/ use homebrew for Mac

 *** Only 1 of the above can be used at any 1 time***


#### Interating with MonboDB using a shell (JS)
- available as a command line program , mongo.exe

Some useful commands to run on MongoDB shell ( NOT tested in exam)
- `help` : get the available shell commands
- `show dbs` : show the currently available databases in MongoDB
- `use <db_name>` : set current database to `<db_name>`
- `db.createCollection(<collection_name>)` : create collection named `<collection_name>` in the database
- after you have set your current database, you can insert documents into the database by running `db.<collection_name>.insert(<json_obj>)`
- `show collections` : show the available collections in the current database
- `db.<collection_name>.find()`: show the documents in the collection

> Instead of creating collection with `db.createCollection(<collection_name>)`, `db.<collection_name>.insert(<json_obj>)` will automatically create the collection with the document is added.

### Exercise 1
On MongoDB shell, create a database called `test_info` and insert the following JSON object as a document in the collection `Person` in the database.

>```python
>{
> 'name':'John Lim',   
> 'class': '18S01'   
>}
>```


Query by using:
```
db.test_info.find()
````

### Interacting with MongoDB with `pymongo`

Similar to relational databases, we need to know how to execute the important database operations (CRUD) with MongoDB as well. However, for MongoDB, we will skip on the MongoDB shell commands and go straight up to the commands in `pymongo`, which is a Python module to interact with MongoDB databases (as warned earlier, keep the MongoDB running else you will encounter errors.)

#### Connecting to MongoDB database with `pymongo`
To work with the database,
1. We first **establish connection** to the MongoDB server by creating `pymongo.MongoClient` object to `localhost` with the default port `27107`
2. Access the database through the client.
3. Access the collection through the database.
4. Perform  CRUD operations on the collection.

Working with database, colection and document from PyMongo

In [1]:
import pymongo
client = pymongo.MongoClient("localhost", 27017)# client object

print(f"DBs:\n{client.list_database_names()}" )

db = client.get_database("Entertainment") #db object
print(f"Collections in {db.name}\n:{db.collection_names()}")

col = db.get_collection("movies") #collection object

ModuleNotFoundError: ignored

In [ ]:
## print all the documents in the movies collection in the Entertainment DB
list( col.find() )


In [ ]:
## Get the list of all movie names using 1 line
col = pymongo.MongoClient('localhost', 27017)['Entertainment']['movies']

#### CRUD operations with `pymongo`
Unlike `sqlite` which do CRUD operations by passing SQL statements into the `execute` command, the CRUD operations with `pymongo` is done through various methods to the objects found in `pymongo`. Most of the methods act on  `pymongo.collection.Collection` objects and they are:
- `insert_one()` : insert one document into a collection
- `insert_many()` : insert more than one document into a collection
- `find()` : to query documents from the collection
- `update_one()` : to update a document in the collection
- `update_many()` : to update more than one document in the collection
- `delete()`
- `drop()`

#### INSERT

In [ ]:
from pymongo import MongoClient

client = MongoClient('localhost', 27017) #localhost is your local computer address 127.0.0.1
#client.drop_database('test_info') ## reset
db = client['test_info'] # where <DATABASE_NAME> should be replaced with appropriate string
coll = db['Person']

## crud operation are methods in a collection object
## arguments are all dictionary objects/ list of dictionary objects


ModuleNotFoundError: ignored

In [ ]:
list(coll.find())
db.drop_collection('Person')
coll = db['Person']
coll.insert_one(
    {
        "name":"Joe",
        "sur_name":None,
        "age": None
    }
)

In [ ]:
list(coll.find())

In [ ]:
extra_persons = [{
    'name':'Ben',
    'age': '15',
    'hobbies': ['running','reading','gaming']
},{
    'name':'Lim Bo',
    'class': '18S01',
    'hobbies': ['gaming']
}
]

coll.insert_many(extra_persons)

print(list(coll.find()))

#### Exercise 2
load the students info in students.csv into the Person collection in test_info database

In [ ]:
## Code here

from pymongo import MongoClient

client = MongoClient('localhost', 27017)
db = client['test_info']
header = open("students.csv","r").read().strip().split('\n')[0].split(',')
l = open("students.csv","r").read().strip().split('\n')[1:]
db.drop_collection('Person')
coll = db['Person']
for i in l:
    temp = i.split(",")
    coll.insert_one(
        {
            header[0]:temp[0], ## read from 1st row in file instead of hardcoding
            header[1]:temp[1],
            header[2]: temp[2]
        }
    )
print(list(coll.find()))

In [ ]:
#Kavish
#Insert Many

col = pymongo.MongoClient("mongodb://localhost:27017/")["testdb"]["Person"]
csv = [item.split(",") for item in open("students.csv").read().strip().split("\n")]
students = [
    {
        "name": item[2],
        "class": item[0],
        "index": item[1]
    }
    for index, item in enumerate(csv) if index != 0
]
col.delete_many({})
col.insert_many(students)
print(list(col.find()))

NameError: ignored

##### Example Using JSON

In [ ]:
import json, pymongo
#get the data from file
#insert data into collection
pymongo.MongoClient("localhost", 27017).drop_database("test_info")

pymongo.MongoClient("localhost", 27017)["test_info"]["Person"].insert_many(
json.load(open("students.json"))
)


____
#### Query Documents in a collection [READ]
```
- <collection>.find(<query>, <projection>)  
- <collection>.find()
    #SELECT * FROM person

- <collection>.find({"name":"John"})
    #SELECT * FROM person WHERE name=  “John”
```
- query is a python dict with the attributes' values you want to retrieve
- result is a iterator of python dict objects

- Projection
```
db.person.find({ }, {"name":1,"_id:0})  
    SELECT name FROM person
```

In [ ]:
import pymongo
col = pymongo.MongoClient('localhost', 27017)['test_info']['Person']
print( list(col.find_one ()))


ModuleNotFoundError: ignored

##### using operators in the query filter
```
#implicit AND
<collection>.find({"name": "John", "class":"18S01"})
#SELECT * FROM person WHERE name=  “John” AND class = "18S01"

#explicit OR
<collection>.find( {"$or": [  { "name":"John"}, {"class":"18S02"  }  ] } )
#SELECT * FROM person WHERE name=  “John” OR class = "18S02"

```
<center>

| ** Logical Logical Operator** | **Description** |
|-|-|
| `$and` | Joins query clauses with a logical `AND` returns all documents that match the conditions of both clauses. |
| `$or ` | Joins query clauses with a logical OR returns all documents that match the conditions of either clause. |


</center>

In [ ]:
import pymongo
col = pymongo.MongoClient('localhost', 27017)['test_info']['Person']
## Code : note that the query with operator is a postfix expression
query = {"$or": [ {"name":"John"}, {'class': "18S02"}] }

projection = {
    "name": 1,
    "class": True,
    "_id": 0
}
list(col.find(query, projection))


In [ ]:
col = pymongo.MongoClient('localhost', 27017)['test_info']['Person']
## Code : note that the query with operator is a postfix expression
query = {"$or": [ {"name":"John"}, {'class': "18S02"}] }
projection = {
    "name": 1,
    "class": True,
    "_id": 0
}
list(col.find(query, projection))

In [ ]:
## Code : using Python to filter and get data
col = pymongo.MongoClient('localhost', 27017)['test_info']['Person']
ret = []
for doc in col.find({}, {'name':1,'class':1,'_id':0} ):
    if doc['name'] == 'John' or doc['class'] == "18S02":
        ret.append(doc)
print(ret)


#### Query a nested attribute

Example : Find all the students whose PM is Mr Chan

In [ ]:
## Code here
query = {
    "PM.name":"Chan"
}
list(col.find(query, projection))

#### Using comparison operators in an attribute value
Example : Find all PM's younger than 30


In [ ]:
query = {
    "PM.age": {"$lt": 30}
}
projection = {
    "name": 1,
    "class": True,
    "_id": 0,
    "PM.age":1,
    "PM.name":1
}
list(col.find(query, projection))


| **Comparison Operator** | **Description** |
|-|-|
| `$eq` | Matches values that are equal to the given value. |
| `$gt` | Matches if values are greater than the given value. |
| `$lt` | Matches if values are less than the given value. |
| `$gte` | Matches if values are greater or equal to the given value. |
| `$lte` | Matches if values are less or equal to the given value. |
| `$in` | Matches any of the values in an array. |
| `$ne` | Matches values that are not equal to the given value. |
| `$nin` | Matches none of the values specified in an array. |
| `$exists` | Presence of a attribute/field. |

</center>

<br>

You can head to <a href='https://docs.mongodb.com/manual/reference/operator/query/'>the official query operator docs</a> for more examples and operators. However, the ones mentioned above should suffice for most cases.

### Exercise 3
In the `Person` collection in the `test_info` database,

Task 0. print out all the documents in the collection without showing the _id

Task 1. find the documents with 'class' not equal '18S01'

Task 2. find the documents whose class is 18S01 and PM's age is > 30

Task 3. find all student names who has gaming as a hobby

Task 4. find all student names who has gaming or bowling as a hobby

Task 5 find all student names who do not have any hobby

In [ ]:
#Task 0
col = pymongo.MongoClient('localhost', 27017)['test_info']['Person']
list(col.find({},{'_id':0}))

In [ ]:
#Task 1 zeyu
import pymongo
col = pymongo.MongoClient('localhost', 27017)['test_info']['Person']
query={
    "class": {'$ne': '18S01'}
}
list(col.find(query))

In [ ]:
# Task 2 ta wenn
import pymongo
col = pymongo.MongoClient('localhost', 27017)['test_info']['Person']
query = {
"class": {"$eq":"18S01"},
"PM.age": {"$gt":30}
    }
proj = {
"_id": 0
    }
print(list(col.find(query, proj)))

In [ ]:
# Task 3 Zi Zhuo
import pymongo
col = pymongo.MongoClient('localhost', 27017)['test_info']['Person']
query = {
    'hobbies':{'$in':['gaming']}
    }
projection = {'_id':0,
              'name':1}

print(list(col.find(query,projection)))

In [ ]:
# Task 3
import pymongo
col = pymongo.MongoClient('localhost', 27017)['test_info']['Person']
query1 = {
    'hobbies':'gaming'
    }
query2 = {
    'hobbies':'bowling'
    }
query3 = {'$and':[query1,query2]}

projection = {'_id':0,
              'name':1}

print(list(col.find(query3,projection)))

In [ ]:
# Task 4
import pymongo
col = pymongo.MongoClient('localhost', 27017)['test_info']['Person']
query = {
    'hobbies':{'$in':['gaming','bowling']}
    }
projection = {'_id':0,
              'name':1}

print(list(col.find(query,projection)))

In [ ]:
# Task 5 (Jayden)
import pymongo
col = pymongo.MongoClient('localhost', 27017)['test_info']['Person']
query = {
    'hobbies':{'$exists':0}
}
proj = {
    '_id':0
}
list(col.find(query,proj))

#### Updating Documents in a collection [UPDATE]

1. Update one document

    - use
    
    `coll.update_one(<my_query>,<my_values>)`, where
    ```
    <my_values>=
    {'$set':
        {
            <attribute_name_1>:<value_1>,
            <attribute_name_2>:<value_2>,...
        }
    }
    ```
    > Take note that if an attribute does not exits in the current document, the attribute, with its value will be created
    
2. Update many documents

    - update all documents satisfying your query, use `coll.update_many(<my_query>,<my_values>)`.

3. `update_one()` and `update_many()` methods accept a Boolean parameter `upsert` (default is `False`) which modify their behaviours a little bit. If `upsert=True` and the query does not match any documents in MongoDB, the method will perform an insertion of the document instead. Otherwise, if the query does not match any documents, no insertion of such records. This approach could be handy if you want to avoid using conditionals to handle such cases.

Example:
```
db.person.updateOne({“name”:"John"},
                    {$set:{
                        “name”:"NewJohn","class":"19S01"
                        }
                    } )
# UPDATE person SET name =“NewJohn”,class="19S01"  WHERE name=“John”
```


### Exercise 4
In the `Person` collection in the `test_info` database, update all the documents with PM.name = "Chan"  set the PM.age field for such documents to `50`.

In [ ]:
#YOUR_CODE_HERE
query = {
    'PM.name':'Chan'
}
update = {
    '$set': {
        'PM.age': 50,
        'age':50
    }
}
col.update_many(query, update)


#### Exercise 4A

Update those students who has 'gaming' as a hobby to 'programming'

In [ ]:
query = {"hobbies": "gaming"}
update = {
    "$set":{ "hobbies.$": "programming"}
}
col.update_many(query, update)

#### Delete Documents in a collection [DELETE]

To:
- delete one document satisfying your query, we use `coll.delete_one(<my_query>)`
- delete all documents satisfying your query, we use `coll.delete_many(<my_query>)`
- delete all documents in the collection, we pass empty query `{}` in the `coll.delete_many()` method.

### Exercise 5
In the `Person` collection in the `test_info` database,

- delete all documents from `test_info` database and verify that the database is empty.
- remove the Person collection

In [ ]:
#YOUR_CODE_HERE
col.delete_many({})


In [ ]:
f = open("hammer_bros.jpeg", "rb")
data = f.read()
import pymongo
col = pymongo.MongoClient('localhost', 27017)['bin']['file']
col.insert_one(
    {
    "jpeg": data
    }
)

In [ ]:
from PIL import Image
import io
col = pymongo.MongoClient('localhost', 27017)['bin']['file']
doc = col.find_one({"jpeg": {"$exists":1} })
data = doc["jpeg"]
image = Image.open( io.BytesIO(data))
image.show()

____
#### Exercise 6 2021/DHS/P2/Q4 H2 Computing

### Task 1
Write a program to help staff of an events company to insert data into a NoSQL database products under the collection balloons.

The data is provided for you in `balloons.json` as well as in the table below where the first row are headers for the fields.

<center>

| `design` | `amount` | `helium` | `colours` |
|-|-|-|-|
| car | 88 | no | red, yellow |
| cloud | 14 | | blue, green |
| flower | 75 | yes | red, blue |
| bag | 38 | no | red, blue, black |

</center>

Each colour in `colours` field should be an item in an array. <div style="text-align: right">[6]</div>

In [ ]:
#YOUR_CODE_HERE


### Task 2
Write code to print the `amount` of the product with the design "`car`". <div style="text-align: right">[2]</div>

In [ ]:
#YOUR_CODE_HERE

### Task 3
Write code to update the field helium to have the value "no" for all documents which do not have a field or value for helium. <div style="text-align: right">[3]</div>

In [ ]:
#YOUR_CODE_HERE

### Task 4
Write code to display the design(s) which do not contain helium and have colours that either contain green or do not contain black. <div style="text-align: right">[3]</div>
Run the program.

____
### Appendix
##### JSON representation of Python objects

In [ ]:
import json
## string representaion of a dictionary

L =[{1.2:"23"},2,3,"abc",(8,9)]

ret = json.dumps(L) ## all keys are converted to str
print(ret)

LL = json.loads(ret)
print(LL, type(LL))

##### Using regular expression to match format of string
```
* 0 or more
+ 1 or more
?  0 or 1
| or
- to
. any character
^ starts with
$ ends with
[] any 1 of the following
{min,max} number of times,
() group of characters
```


In [ ]:
col = pymongo.MongoClient('localhost', 27017)['test_info']['Person']
re = {"name": {'$regex':'^Jo*'}}
projection = {
    "name": 1,
    "class": True,
    "_id": 0,
    "PM.name":1
}
list(col.find(re, projection))
